In [204]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [214]:
# pour chaque fichier csv dans ./datas, afficher le shape du dataframe
import os
import pandas as pd

for file in os.listdir('./cleaned_datas'):
    if file.endswith('.csv'):
        df = pd.read_csv(f'./cleaned_datas/{file}')
        print(df.shape)

(70063, 16)
(60490, 16)
(56707, 16)


In [205]:
import pandas as pd
import numpy as np
import ast
import re

def extract_bathroom_value(bathrooms_text):

    if pd.isnull(bathrooms_text):
        return None
    bathrooms_text = bathrooms_text.lower()
    if "half-bath" in bathrooms_text:
        return 0.5
    match = re.search(r"(\d*\.?\d+)", bathrooms_text)
    return float(match.group(1)) if match else None

def generate_bathrooms_text(bathrooms_value):
    """
    Génère une valeur pour 'bathrooms_text' à partir de 'bathrooms'.
    Formate comme '<valeur> bath'.
    """
    if pd.isnull(bathrooms_value):
        return None
    if bathrooms_value == 0.5:
        return "Half-bath"
    return f"{bathrooms_value} bath"

def csv_to_dataframe(path):
    """
    Charge un fichier CSV, nettoie les colonnes 'bathrooms' et 'bathrooms_text', et sauvegarde un fichier nettoyé.
    """
    # Extraction du nom de fichier sans extension
    df_name = path.split('/')[-1].split('.')[0]
    df = pd.read_csv(path)

    # Afficher les types de données pour référence
    print(df.dtypes.to_dict())

    # Colonnes à conserver
    columns_to_keep = [
        "listing_url",
        "bathrooms",
        "bedrooms",
        "accommodates",
        "bathrooms_text",
        "beds",
        "property_type",
        "room_type",
        "latitude",
        "longitude",
        "neighbourhood_cleansed",
        "minimum_nights",
        "maximum_nights",
        "availability_365",
        "instant_bookable",
        "price"
    ]
    df = df[columns_to_keep]

    # Supprime les lignes ou bds est null
    df = df[~df['beds'].isnull()]

    # Supprimer les lignes où 'bathrooms' et 'bathrooms_text' sont toutes deux nulles
    df = df[~df[['bathrooms_text', 'bathrooms']].isnull().all(axis=1)]

    # Remplir les valeurs nulles de 'bathrooms' en utilisant 'bathrooms_text'
    df.loc[df['bathrooms'].isnull(), 'bathrooms'] = df.loc[df['bathrooms'].isnull(), 'bathrooms_text'].apply(extract_bathroom_value)

    # Remplir les valeurs nulles de 'bathrooms_text' en utilisant 'bathrooms'
    df.loc[df['bathrooms_text'].isnull(), 'bathrooms_text'] = df.loc[df['bathrooms_text'].isnull(), 'bathrooms'].apply(generate_bathrooms_text)

    # retirer le dollas signe de la colonne price
    df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

    # Sauvegarder le DataFrame nettoyé
    cleaned_path = f'./cleaned_datas/{df_name}_cleaned.csv'
    df.to_csv(cleaned_path, index=False)

    print(f"Cleaned data saved to: {cleaned_path}")
    return df


In [206]:

def fill_missing_prices(df):
    columns_to_match = [
        "bathrooms",
        "bedrooms",
        "accommodates",
        "beds",
        "property_type",
        "room_type",
    ]

    # Fonction pour calculer le prix médian des lignes similaires
    def find_median_price(row):
        # Si le prix n'est pas manquant, on ne fait rien
        if not pd.isnull(row['price']):
            return row['price']

        # Filtrer les lignes similaires
        similar_rows = df.copy()
        for col in columns_to_match:
            if not pd.isnull(row[col]):
                similar_rows = similar_rows[similar_rows[col] == row[col]]

        print(f"Found {len(similar_rows)} similar rows for {row['listing_url']}")

        # Calculer le prix médian des lignes similaires ayant un prix non nul
        median_price = similar_rows['price'].dropna().median()

        # Retourner le prix médian (ou NaN si aucune ligne similaire n'a un prix)
        print(f"Found median price {median_price}")
        return median_price

    # Appliquer la fonction pour remplir les prix manquants
    df['price'] = df.apply(find_median_price, axis=1)

    return df

df = csv_to_dataframe('./datas/airbnb_paris_september_2024.csv')
df = fill_missing_prices(df)
df.to_csv('./cleaned_datas/airbnb_paris_september_2024_cleaned.csv', index=False)


{'id': dtype('int64'), 'listing_url': dtype('O'), 'scrape_id': dtype('int64'), 'last_scraped': dtype('O'), 'source': dtype('O'), 'name': dtype('O'), 'description': dtype('O'), 'neighborhood_overview': dtype('O'), 'picture_url': dtype('O'), 'host_id': dtype('int64'), 'host_url': dtype('O'), 'host_name': dtype('O'), 'host_since': dtype('O'), 'host_location': dtype('O'), 'host_about': dtype('O'), 'host_response_time': dtype('O'), 'host_response_rate': dtype('O'), 'host_acceptance_rate': dtype('O'), 'host_is_superhost': dtype('O'), 'host_thumbnail_url': dtype('O'), 'host_picture_url': dtype('O'), 'host_neighbourhood': dtype('O'), 'host_listings_count': dtype('float64'), 'host_total_listings_count': dtype('float64'), 'host_verifications': dtype('O'), 'host_has_profile_pic': dtype('O'), 'host_identity_verified': dtype('O'), 'neighbourhood': dtype('O'), 'neighbourhood_cleansed': dtype('O'), 'neighbourhood_group_cleansed': dtype('float64'), 'latitude': dtype('float64'), 'longitude': dtype('flo

In [207]:
# afficher les doublons
duplicates = df[df.duplicated()]
print(duplicates)


Empty DataFrame
Columns: [listing_url, bathrooms, bedrooms, accommodates, bathrooms_text, beds, property_type, room_type, latitude, longitude, neighbourhood_cleansed, minimum_nights, maximum_nights, availability_365, instant_bookable, price]
Index: []


In [208]:
df.shape[0]

64083

In [209]:
# Afficher les colonnes avec des données manquantes
missing_data = df.isnull().sum()
missing_data

listing_url                 0
bathrooms                   0
bedrooms                  122
accommodates                0
bathrooms_text              0
beds                        0
property_type               0
room_type                   0
latitude                    0
longitude                   0
neighbourhood_cleansed      0
minimum_nights              0
maximum_nights              0
availability_365            0
instant_bookable            0
price                       0
dtype: int64

In [210]:
# dropna la ou price isna()
df = df.dropna(subset=['price'])
df = df.dropna(subset=['bedrooms'])


In [211]:
df = df[df['room_type'] != 'Hotel room']
df = df[df['property_type'] != 'Room in hotel']
df = df[df['bathrooms'] != 20]
df = df[df['bedrooms'] != 32]
df = df[df['bedrooms'] != 14]
df = df[df['price'] <= 1000]
df = df[df['maximum_nights'] <= 1460]
df = df[df['minimum_nights'] <= 90]

df.to_csv('./cleaned_datas/airbnb_paris_september_2024_cleaned.csv', index=False)
df.describe()

,bathrooms,bedrooms,accommodates,beds,latitude,longitude,minimum_nights,maximum_nights,availability_365,price
count,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000,60490.000000
mean,1.175748,1.281253,3.327525,1.757464,48.863768,2.342117,6.301438,399.811704,179.719954,206.333675
std,0.455361,0.831186,1.672114,1.129798,0.018118,0.034574,12.115719,394.834639,119.182830,162.153524
min,0.000000,0.000000,1.000000,0.000000,48.816090,2.228970,1.000000,1.000000,0.000000,8.000000
25%,1.000000,1.000000,2.000000,1.000000,48.850489,2.319505,1.000000,40.000000,70.000000,101.000000
50%,1.000000,1.000000,3.000000,1.000000,48.864980,2.346211,2.000000,365.000000,177.000000,150.000000
75%,1.000000,2.000000,4.000000,2.000000,48.878306,2.366911,4.000000,365.000000,276.000000,250.000000
max,7.500000,13.000000,16.000000,29.000000,48.901590,2.468360,90.000000,1126.000000,365.000000,1000.000000
